# CSE 204 Lab 13: Reinforcement Learning II

Jesse Read, J.B. Scoggins

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jbscoggi/teaching/blob/master/Polytechnique/CSE204/Lab13_answers.ipynb) 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jbscoggi/teaching/master?filepath=Polytechnique%2FCSE204%2FLab13_answers.ipynb)

## Introduction

This is the second lab dealing with reinforcement learning.  Today, the goal will be to implement and train an agent to balance a pole on top of a moveable cart using the hill climbing algorithm discussed in the lecture.  We will again use the [OpenAI Gym](https://gym.openai.com) library that you saw in the last lab for creating the environment for our agent to act in.  To get started, import the necessary packages.

In [ ]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

import numpy as np
from numpy.random import choice as sample
from numpy.random import rand, randn

import gym
import matplotlib.pyplot as plt


Next, just run the cell below which provides utility functions for 

In [ ]:

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)"
https://star-ai.github.io/Rendering-OpenAi-Gym-in-Colaboratory/
"""

from gym import logger as gymlogger
gymlogger.set_level(40) #error only
from gym.wrappers import Monitor

import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
# Only in colab!
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

Before proceeding with the excercise, you can run the following cell to see the cart-pole problem in action with an agent that takes random actions at each time step.

In [ ]:
env = wrap_env(gym.make('CartPole-v1'))
env.reset()
for _ in range(200):
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        env.reset()
env.close()
show_video()

Unless you got very lucky, you will notice that the pole on top of the cart fell and possibly spun around the cart several times.  In fact, this problem is physically unstable, and without a proper controller, the pole will always fall.  In order to stop the pole from falling, the agent is given two possible actions: 1) apply a positive force +1 or 2) apply a negative force -1.  The response to this force depends on the friction of the cart on the track, and the pole on the cart, the mass of both the cart and the pole, and the horizontal and angular velocities and the cart and pole respectively.  At any given point in time, the state of the system is fully characterized by 4 variables:

1. The horizontal location of the cart
2. The horizontal velocity of the cart
3. The angular position of the pole
4. The angular velocity of the pole

Given those four quantities and the force imposed on the cart, the system behavior is governed by a set of differential equations, which are hidden from you and the agent.  Your task therefore, is to build an agent which learns from the response of the cart-pole system to various force inputs, such that it can create a policy for balancing the pole.

## Hill Climbing

As you may recall, hill climbing is a simple but powerful learning technique.  Here is a quick refresher if you don't remember the details.  First, let's recall a bit of the terminology.

1. We have an environment (in this case the cart-pole problem).
2. An agent maps observations of the state of the environment $\mathcal{S}$ to allowed actions $\mathcal{A}$ through a policy $\pi : \mathcal{S} \mapsto \mathcal{A}$.
3. A reward function provides a metric to measure the benifit of taking an action for a given state.
4. Ideally, the best policy is one that maximizes the return, which is the sum of the rewards for a series of actions over a finite amount of time.
5. A value function maps a given state to the expected return given the best policy.

In this lab, we will work with policy learning, which means that we will search for the best policy that maximizes the return value.  Specifically, we consider policies of the form

$$ a_t = \pi_\theta(s_t) = \text{argmax} \;\sigma(W^T s_t + b) $$

where $a_t$ is the action at time $t$, given the policy and state at time $t$.  The policy parameters $\theta = \{W, b\}$ represent the weights and biases for the linear transformation of the state space into action space, and $\sigma$ is a nonlinear operator, such as the sigmoid function.  Note this is just a single layer perceptron, like the ones you have dealt with in previous labs on neural networks.  The difference here, however, is that we will not train this network with gradient descent, but rather with the hill climbing algorithm, which is summarized as follows.

---
**Algorithm 1: Random Optimization in Continuous Search Space (Hill Climbing)**
1. Initialize weights and biases, $\theta$
2. Select minimal return desired, $g_{\text{min}}$.
3. Loop while $g(\theta) < g_{\text{min}}$
   1. Sample nearby parameters, $\theta' \sim N(\theta, \text{variance})$
   2. if $g(\theta') > g(\theta)$:
      1. Update parameters, $\theta \gets \theta'$
---

In the algorithm above, $g(\theta)$ is the return, i.e. the sum of rewards for a series of actions taken with the same policy ($\theta$).  Note also that the perturbation variance is a hyperparameter in our algorithm.  Finally, we can modify the algorithm above by taking random resets during sampling.  In other words, take a random sample centered at zero, rather than at the current parameter vector, every so often.  This helps to eliminate stalling and poor convergence. 


## Part 1: Implement policy function

**Exercise 1.1**: Fill in the code below to implement the SLP policy function.

In [ ]:
def linear(a):
    '''Linear activation.'''
    return a

def sigmoid(a):
    '''Sigmoid activation.'''
    return 1.0 / (1.0 + np.exp(-a))

class SLP():
    '''A Single-Layer Perceptron with copy and random modification.'''

    def __init__(self, inputs, outputs, activation = linear):
        self.input_size = inputs
        self.output_size = outputs
        self.reset()
        self.activation = activation

    def reset(self, scaling=0.0):
        '''Resets the weights and biases to normal distribution with scaling.'''
        # TODO
        self.W = 
        self.b = 

    def predict(self,x):
        '''Feedforward prediction of SLP.'''
        # TODO
        return ?

    def copy(self, modify=False):
        '''Creates a copy of this SLP, with possible modification.'''
        
        # Create a copy
        b = SLP(self.input_size, self.output_size)
        b.W = self.W.copy()
        b.b = self.b.copy()

        if modify:
            b.modify()

        return b

    def modify(self, alpha_w=0.01, alpha_b=0.01, prob_reset=0.1):
        '''Adds perturbation to weights and biases, scaled by alphas, with random reset probability.'''

        if rand() < prob_reset:
            self.reset()

        # Make a random adjustment to the weight matrix and bias vector.
        # TODO
        self.W = 
        self.b = 

## Part 2: Implement the SimpleHillClimber Agent

In this part, we will implement a simple hill climber.  The agent has two functions, 1) update the policy given an observation and reward and 2) act on an observation (evaluate the policy).  To help get you started, the next cell has the code which we will use to place our agent in the environment.

In [ ]:
def run_agent(env, agent, num_epsiodes = 50 , max_episode_length=200, train=False):

    R = 0 

    # For n episodes ...
    for i in range(num_epsiodes):

        s_t = env.reset()

        # For each time step ...
        for t in range(max_episode_length):

            # Draw
            if not train:
                env.render()

            # Act
            a_t = agent.act(s_t, training=train)

            # Step
            s_t, r_t, done, info = env.step(a_t)

            # Update
            agent.update_policy(s_t, r_t, done)

            R = R + r_t
            if done:
                break
    
    env.close()

    return (1.0 / num_epsiodes) * R

The outer loop in the code above loops over the number of "episodes." An episode is a single simulation from beginning to end, starting from a clean environment.  The inner loop is looping over the maximum number of iterations in each episode.  Note that it is possible for the environment to decide that the episode is complete before the maximum iterations is reached.  This may happen if the cart or pole leave some bounded domain, for example.  A single iteration is controlled by 4 steps, as shown in the code:

1. Render the environment (optional)
2. Ask the agent for an action given the current state of the system (observation)
3. Simulate a single step in the environment (perform the action)
4. Allow the agent to update its policy

At the end of all the episodes, the function returns the average reward per episode.  Note that, for the cart and pole environment, rewards are either +1 when the pole does not fall below 15 degrees from vertical in a single step and zero otherwise.  Therefore, if we take 200 steps per episode, the best average reward we can obtain is 200.

**Exercise 2.1**: Implement the SimpleHillClimber class based on Algorithm 1.  The agent will keep track of two policies, 1) the best policy so far, and 2) a random perturbation from the best policy.  

At each call to update_policy, the agent will update the current return (sum of rewards) and total number of calls.  
If the end of an episode is reached, the current value of the return should be added to a "memory" vector, and return and number of iterations should be set back to zero.   
Once the number of episodes per test is reached, the agent should check if the average return per episode is improved over the current policy.  If it is, then the current policy is replaced by the trial policy.  Either way, a new trail policy should then be created using the modify function of the SLP policy and the memory vector should be reset to empty.

Whenever act is called, the current trial policy should be evaluated and returned if training, otherwise the current policy should be used.

In [ ]:
class SimpleHillClimber():
    '''Simple Hill Climbing Agent'''

    def __init__(self, obs_space, act_space, max_episode_length=50, num_episodes_per_test=100, alpha=0.1):
        # TODO

    def update_policy(self,obs,reward,done=False):
        # TODO
            

    def act(self,obs,reward=None,done=False,training=False):
        # TODO

**Exercise 2.2**: Run the training loop below.  You should see the average reward increasing as the number of episodes increases.  Note, that since we are using a rather simple algorithm here, you may need to restart this a few times to get a good result.  If you have tried it 3-5 times with no improvement, then you may have a bug in your hill climber agent.

In [ ]:
env = wrap_env(gym.make('CartPole-v1'))
env.reset()
agent = SimpleHillClimber(
    env.observation_space, env.action_space, 
    max_episode_length=200, num_episodes_per_test=100, alpha=0.1)

t = 0
average_returns = [0,0,0,0,0,0,0,0,0,0,0]
while np.min(average_returns[-10:-1]) < 150:

    # Run the agent in the environment
    R = run_agent(env, agent, train=True)

    # Monitoring
    average_returns.append(R)
    t += 1

    print("Episode {0}, average return: {1}".format(t*50, R))

plt.plot(range(len(average_returns)),average_returns)
plt.title("Average reward on 100 episodes")
plt.xlabel("Training Steps")
plt.ylabel("Reward")
plt.show()

**Exercise 2.3**: Run the agent for a single episode to see an animation of your trained agent.

In [ ]:
# Finished, show animation; training curve
env = wrap_env(gym.make('CartPole-v1'))
env.reset()
R = run_agent(env, agent, num_epsiodes=1, train=False)
show_video()
print(R)


## Going further

There are a number of improvements that you can make to the simple hill climber we made here.  For example, you can use simulated annealing or beam search which you saw in the last lecture.  If you have time, consider adding one or more of these improvements.  Finally, the agent and code we created above is completely general.  Therefore you can try running on other environments.  For a complete list, look [here](https://gym.openai.com/docs/#available-environments).